In [1]:
import re
import logging

import numpy as np
import pandas as pd
import spacy
import torch
import csv
from torchtext import data
import urllib.request

In [2]:
#The User data 
User_data = pd.read_csv('data/user-info.csv', usecols = ['user.id','doc.id','rating'])
User_data = User_data.rename(columns={'user.id': 'user_id','doc.id': 'doc_id'})


In [3]:
# The article dataset
article_data = pd.read_csv('data/raw-data.csv', usecols = ['doc.id','title','citeulike.id', 'raw.abstract'],encoding = "ISO-8859-1")
article_data = article_data.rename(columns={'raw.abstract': 'abstract','doc.id': 'doc_id','citeulike.id': 'citeulike_id'})


In [4]:
CiteULike_data=pd.merge(User_data,article_data,on="doc_id")

In [5]:
VAL_RATIO = 0.2

def prepare_csv(seed=999):
    df_train = CiteULike_data
    df_train["abstract"] = \
        df_train.abstract.str.replace("\n", " ")
    idx = np.arange(df_train.shape[0])
    np.random.seed(seed)
    np.random.shuffle(idx)
    val_size = int(len(idx) * VAL_RATIO)
    df_train.iloc[idx[val_size:], :].to_csv(
        "cache/dataset_train.csv", index=False)
    df_train.iloc[idx[:val_size], :].to_csv(
        "cache/dataset_val.csv", index=False)

In [6]:
prepare_csv(999)

In [7]:
le = max(len(x) for x in CiteULike_data.abstract)
print(le)

122938


In [8]:
NLP = spacy.load('en')
MAX_CHARS = 1229381
def tokenizer(abstract):
    comment = re.sub(
        r"[\*\"“”\n\\…\+\-\/\=\(\)‘•:\[\]\|’\!;]", " ", 
        str(abstract))
    abstract = re.sub(r"[ ]+", " ", str(abstract))
    abstract = re.sub(r"\!+", "!", str(abstract))
    abstract = re.sub(r"\,+", ",", str(abstract))
    abstract = re.sub(r"\?+", "?", str(abstract))
    if (len(abstract) > MAX_CHARS):
        abstract = abstract[:MAX_CHARS]
    return [
        x.text for x in NLP.tokenizer(abstract) if x.text != " "]

In [13]:
len(CiteULike_data.doc_id.unique())

16980

In [21]:
#CiteULike_data.abstract=CiteULike_data.abstract.to_string()

In [22]:
#print(type(CiteULike_data.abstract))

In [14]:
print((CiteULike_data.abstract))

0         The Nucleic Acids Research Molecular Biology D...
1         The Nucleic Acids Research Molecular Biology D...
2         The Nucleic Acids Research Molecular Biology D...
3         The Nucleic Acids Research Molecular Biology D...
4         The Nucleic Acids Research Molecular Biology D...
5         The Nucleic Acids Research Molecular Biology D...
6         The Nucleic Acids Research Molecular Biology D...
7         The Nucleic Acids Research Molecular Biology D...
8         The Nucleic Acids Research Molecular Biology D...
9         The Nucleic Acids Research Molecular Biology D...
10        The Nucleic Acids Research Molecular Biology D...
11        The Nucleic Acids Research Molecular Biology D...
12        The Nucleic Acids Research Molecular Biology D...
13        The Nucleic Acids Research Molecular Biology D...
14        The Nucleic Acids Research Molecular Biology D...
15        The Nucleic Acids Research Molecular Biology D...
16        The Nucleic Acids Research Mol

In [15]:
len(tokenizer(CiteULike_data.abstract))

641

In [28]:
len(CiteULike_data.abstract)

204986